In [12]:
import time
import requests

from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the GitHub token from an environment variable
token = os.getenv('GITHUB_TOKEN')

if not token:
    raise Exception("Please set the GITHUB_TOKEN environment variable in the .env file")
project = {
    'authors': set(),
    'commit_count': 0,
    'without_commits': set()
}

coaches = ['icarito', 'unjust']


def get_forks(github_user, repo):
    url = f"https://api.github.com/repos/{github_user}/{repo}/forks"
    headers = {'Authorization': f'token {token}'}
    response = requests.get(url, headers=headers)
    forks = response.json()
    return forks


def get_commit_stats(github_user, repo):
    url = f"https://api.github.com/repos/{github_user}/{repo}/stats/contributors"
    headers = {'Authorization': f'token {token}'}
    response = requests.get(url, headers=headers)

    # Check if the response status is 202 (Accepted), indicating that the data is being computed
    while response.status_code == 202:
        time.sleep(2)  # Wait for 2 seconds before making another request
        response = requests.get(url, headers=headers)

    # If the status code is not 200 (OK), raise an error
    if response.status_code != 200:
        raise Exception(
            f"Failed to fetch commit stats: {response.status_code}")

    commit_stats = response.json()
    return commit_stats


all_forks = get_forks('Laboratoria', 'UPSK-PY-002-guess-the-number')
forks = [fork for fork in all_forks if fork['owner']['login'] not in coaches]

for fork in forks:
    fork_user = fork['owner']['login']
    fork_name = fork['name']
    print(f"Fork: {fork_name} by {fork_user}")
    student_commits = 0

    commit_stats = get_commit_stats(fork_user, fork_name)
    for stat in commit_stats:
        author = stat['author']['login']
        if author not in coaches:
            project['authors'].add(author)
            author_commits = stat['total']
            student_commits += author_commits
            print(f"Author: {author}, Commits: {author_commits}")
    project['commit_count'] += student_commits
    if student_commits == 0:
        project['without_commits'].add(fork_user)

print(f"Estudiantes sin commits: {project['without_commits']}")

Fork: guess-the-number by gabrielafaria608
Fork: UPSK-PY-002-guess-the-number by anabfer
Fork: UPSK-PY-002-guess-the-number by SaraAnita1
Author: SaraAnita1, Commits: 9
Fork: UPSK-PY-002-guess-the-number by MailaFAP
Author: MailaFAP, Commits: 3
Fork: UPSK-PY-002-guess-the-number by adilamarcelefreitas
Fork: UPSK-PY-002-guess-the-number by jay-santana
Author: jay-santana, Commits: 9
Fork: Guess-the-number-Upsk-Python by ribeirober1208
Estudiantes sin commits: {'gabrielafaria608', 'anabfer', 'ribeirober1208', 'adilamarcelefreitas'}


## Row

In [13]:
#| content: valuebox
#| title: "Forks"
dict(
    icon = "github",
    color = "info",
    value = len(forks),
)

{'icon': 'github', 'color': 'info', 'value': 7}

In [14]:
#| content: valuebox
#| title: "Autores de commits"
dict(
    icon = "person",
    color = "primary",
    value = len(project['authors'])
)

{'icon': 'person', 'color': 'primary', 'value': 3}

In [15]:
#| content: valuebox
#| title: "Commits de Estudiantes"
dict(
    icon = "arrow-up",
    color = "secondary",
    value = project['commit_count']
)

{'icon': 'arrow-up', 'color': 'secondary', 'value': 21}

In [16]:
#| content: valuebox
#| title: "Estudiantes sin commits"
dict(
    icon = "exclamation",
    color = "danger",
    value = len(project['without_commits'])
)

{'icon': 'exclamation', 'color': 'danger', 'value': 4}